# Inadimplência de clientes de cartão de crédito

A base de dados pode ser encontrada em [https://www.kaggle.com/datasets/gabrieloliveirasan/inadimpelncia-de-clientes](https://www.kaggle.com/datasets/gabrieloliveirasan/inadimplncia-de-clientes-de-carto-de-crdito?resource=download)

### Descrição dos dados

**Variáveis preditivas ou explicativas:** Limite de crédito, Sexo (1 = masculino; 2 = feminino), Escolaridade (1 = pós-graduação; 2 = graduação; 3 = ensino médio; 4 = outros), Estado Civil (1 = casado; 2 = solteiro; 3 = outros), Idade, Status de pagamento (Setembro, Agosto, Julho, Junho, Maio, Abril), Valor da fatura (Setembro, Agosto, Julho, Junho, Maio, Abril), Valor do pagamento (Setembro, Agosto, Julho, Junho, Maio, Abril).

Status de pagamento, Valor da fatura e Valor do pagamento, correspondem cada uma a 6 variáveis, sendo que, a variável, Status de Pagamento, recebe os seguintes valores (-1 = Pago devidamente; 1 = Atraso no pagamento por 1 Mês; 2 = Atraso no pagamento por 2 Mês; ...; 9 = Atraso no pagamento por 9 Mês ou Mais)

Mais detalhes sobre a pesquisa podem ser encontrados em 
https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients

A variável resposta foi empregada como sendo, *pagamento inadimplente (Sim = 1, Não = 0)*. Aqui, vamos usar esta variável da seguinte forma.

**Variável resposta ou predita:**  Status Pagamento (1 = Inadimplente, 0 = Adimplente)

A base de dados é de clientes de cartão de crédito com origem em Taiwan, onde a moeda vigente é o Novo dólar taiwanês (NT\$).

## Pré-processamento de dados 

Essa etapa foi realizada em detalhes no notebook Analise exploratória e modelo.

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('credit.csv')
df

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,220000,1,3,1,39,0,0,0,0,...,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29996,29997,150000,1,3,2,43,-1,-1,-1,-1,...,8979,5190,0,1837,3526,8998,129,0,0,0
29997,29998,30000,1,2,2,37,4,3,2,-1,...,20878,20582,19357,0,0,22000,4200,2000,3100,1
29998,29999,80000,1,3,1,41,1,-1,0,0,...,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


In [3]:
df.rename(columns={'ID': 'ID Cliente', 'LIMIT_BAL': 'Limite_Credito','SEX':'Sexo', 'EDUCATION': 'Escolaridade', 
                  'MARRIAGE':'Est_Civil', 'AGE':'Idade', 'PAY_0':'Status_Set', 'PAY_2':'Status_Ago', 
                   'PAY_3':'Status_Jul', 'PAY_4':'Status_Jun', 'PAY_5':'Status_Mai', 'PAY_6':'Status_Abr', 
                   'BILL_AMT1':'Valor_Set', 'BILL_AMT2':'Valor_Ago', 'BILL_AMT3':'Valor_Jul', 
                   'BILL_AMT4':'Valor_Jun', 'BILL_AMT5':'Valor_Mai', 'BILL_AMT6':'Valor_Abr', 
                   'PAY_AMT1':'Pag_Set', 'PAY_AMT2':'Pag_Ago', 'PAY_AMT3':'Pag_Jul', 
                   'PAY_AMT4':'Pag_Jun', 'PAY_AMT5':'Pag_Mai', 'PAY_AMT6':'Pag_Abr', 
                   'default payment next month': 'Status Pagamento'}, inplace = True)

In [4]:
df.loc[df['Est_Civil']==0, 'Est_Civil']=3

df.loc[df['Escolaridade']>4, 'Escolaridade']=4
df.loc[df['Escolaridade']==0, 'Escolaridade']=4

In [5]:
del df['ID Cliente']

# Modelagem 

A modelagem será realizada utilizando a técnica de subamostragem da classe majoritária, essa técnica acaba excluindo observações da classe predominante e pode em alguns caso não ser interessante, mas, avaliando ela em comparação com as demais técnicas, essa foi a que apresentou melhores resultados. 

## Divisão previsões e classe

In [6]:
X_df = df.iloc[:, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22]].values
X_df

array([[ 20000,      2,      2, ...,      0,      0,      0],
       [120000,      2,      2, ...,   1000,      0,   2000],
       [ 90000,      2,      2, ...,   1000,   1000,   5000],
       ...,
       [ 30000,      1,      2, ...,   4200,   2000,   3100],
       [ 80000,      1,      3, ...,   1926,  52964,   1804],
       [ 50000,      1,      2, ...,   1000,   1000,   1000]], dtype=int64)

In [7]:
y_df = df.iloc[:, 23].values
y_df

array([1, 1, 0, ..., 1, 1, 1], dtype=int64)

## Escalonamento de variáveis

In [8]:
from sklearn.preprocessing import StandardScaler
scaler_census = StandardScaler()
X_df = scaler_census.fit_transform(X_df)
X_df[0]

array([-1.13672015,  0.81016074,  0.21186989, -1.06879721, -1.24601985,
        1.79456386,  1.78234817, -0.69666346, -0.66659873, -1.53004603,
       -1.48604076, -0.64250107, -0.64739923, -0.66799331, -0.67249727,
       -0.66305853, -0.65272422, -0.34194162, -0.22708564, -0.29680127,
       -0.30806256, -0.31413612, -0.29338206])

## Divisão entre treinamento e teste

In [9]:
from sklearn.model_selection import train_test_split
seed=1336
np.random.seed(seed)
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size = 0.30, stratify = y_df)

In [10]:
X_train.shape, y_train.shape

((21000, 23), (21000,))

In [11]:
X_test.shape, y_test.shape

((9000, 23), (9000,))

## Técnica de subamostragem da classe majoritária

In [12]:
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
print("Antes: ", Counter(y_train))

Antes:  Counter({0: 16355, 1: 4645})


In [13]:
undersample = RandomUnderSampler(sampling_strategy=1)
X_train, y_train = undersample.fit_resample(X_train, y_train)

In [14]:
print("Depois: ", Counter(y_train))

Depois:  Counter({0: 4645, 1: 4645})


## Modelos de Machine Learning 

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

Observe que utilizo tanto o *predict* - capacidade do modelo de distinguir entre ambas as classes, quanto o *predict_proba* - capacidade de classificar corretamente a classe positiva, e para ambos avalio usando a *Area Under the Curve* (AUC).

### Random Forest

In [16]:
def run_random_forest(X_train, X_test, y_train, y_test):
    rf = RandomForestClassifier(random_state=0, n_estimators=100, max_depth=10, min_samples_split=5, 
                                min_samples_leaf=8, max_features=9)
    rf.fit(X_train, y_train) #treinando o modelo
    score = cross_val_score(rf, X_train, y_train, cv=5, scoring='accuracy')
    print('Resultado Fase de Treinamento')
    print("Pontações da validação cruzada (acurácia) {}".format(score))
    print("Média das pontuação da validação cruzada (acurácia):{}".format(score.mean()))
    
    pred_proba_train = rf.predict_proba(X_train)[:, 1] # Probabilidade da classe positiva para os dados de treino
    print('\nAUC probabilidade (Random Forest) - Treino: {}'.format(roc_auc_score(y_train, pred_proba_train)))
    pred_class_train = rf.predict(X_train) # previsão classe positiva para os dados de treino 
    print('\nAUC previsão (Random Forest) - Treino: {}'.format(roc_auc_score(y_train, pred_class_train)))
    print('Classificação report do Random Forest (Treino):\n {}'.format(classification_report(y_train, pred_class_train)))
    
    print('Resultado Fase de Teste')
    pred_proba_test = rf.predict_proba(X_test)[:, 1] # Probabilidade da classe positiva para os dados de teste
    print('\nAUC probabilidade (Random Forest) - Teste: {}'.format(roc_auc_score(y_test, pred_proba_test)))
    pred_class_test = rf.predict(X_test) # previsão classe positiva para os dados de teste
    print('\nAUC previsão (Random Forest) - Teste: {}'.format(roc_auc_score(y_test, pred_class_test)))
    print('Matriz de confusão AUC (Random Forest) - Teste:\n{}'.format(confusion_matrix(y_test, pred_class_test)))
    print('Classificação report do Random Forest (Teste):\n {}'.format(classification_report(y_test, pred_class_test)))
    return pred_proba_train, pred_class_train, pred_proba_test, pred_class_test

pred_proba_train, pred_class_train, pred_proba_test, pred_class_test = run_random_forest(X_train,X_test,y_train, y_test)

Resultado Fase de Treinamento
Pontações da validação cruzada (acurácia) [0.71582347 0.70452099 0.7088267  0.6883746  0.7131324 ]
Média das pontuação da validação cruzada (acurácia):0.7061356297093648

AUC probabilidade (Random Forest) - Treino: 0.8726247768066637

AUC previsão (Random Forest) - Treino: 0.7781485468245425
Classificação report do Random Forest (Treino):
               precision    recall  f1-score   support

           0       0.74      0.86      0.80      4645
           1       0.84      0.69      0.76      4645

    accuracy                           0.78      9290
   macro avg       0.79      0.78      0.78      9290
weighted avg       0.79      0.78      0.78      9290

Resultado Fase de Teste

AUC probabilidade (Random Forest) - Teste: 0.7826711498647896

AUC previsão (Random Forest) - Teste: 0.7161950205515346
Matriz de confusão AUC (Random Forest) - Teste:
[[5692 1317]
 [ 756 1235]]
Classificação report do Random Forest (Teste):
               precision    recall

### Regressão Logística

In [17]:
def run_Reg_log(X_train, X_test, y_train, y_test):
    rl = LogisticRegression(random_state=0)
    rl.fit(X_train, y_train) #treinando o modelo
    score = cross_val_score(rl, X_train, y_train, cv=5, scoring='accuracy')
    print('Resultado Fase de Treinamento')
    print("Pontações da validação cruzada (acurácia) {}".format(score))
    print("Média das pontuação da validação cruzada (acurácia):{}".format(score.mean()))
    
    pred_proba_train = rl.predict_proba(X_train)[:, 1] # Probabilidade da classe positiva para os dados de treino
    print('\nAUC probabilidade (Regressão Logística) - Treino: {}'.format(roc_auc_score(y_train, pred_proba_train)))
    pred_class_train = rl.predict(X_train) # previsão classe positiva para os dados de treino 
    print('\nAUC previsão (Regressão Logística) - Treino: {}'.format(roc_auc_score(y_train, pred_class_train)))
    print('Classificação report do Regressão Logística (Treino):\n {}'.format(classification_report(y_train, pred_class_train)))
    
    print('Resultado Fase de Teste')
    pred_proba_test = rl.predict_proba(X_test)[:, 1] # Probabilidade da classe positiva para os dados de teste
    print('\nAUC probabilidade (Regressão Logística) - Teste: {}'.format(roc_auc_score(y_test, pred_proba_test)))
    pred_class_test = rl.predict(X_test) # previsão classe positiva para os dados de teste
    print('\nAUC previsão (Regressão Logística) - Teste: {}'.format(roc_auc_score(y_test, pred_class_test)))
    print('Matriz de confusão AUC (Regressão Logística) - Teste:\n{}'.format(confusion_matrix(y_test, pred_class_test)))
    print('Classificação report do Regressão Logística (Teste):\n {}'.format(classification_report(y_test, pred_class_test)))
    return pred_proba_train, pred_class_train, pred_proba_test, pred_class_test

pred_proba_train, pred_class_train, pred_proba_test, pred_class_test = run_Reg_log(X_train,X_test, y_train, y_test)

Resultado Fase de Treinamento
Pontações da validação cruzada (acurácia) [0.67545748 0.66899892 0.66200215 0.66846071 0.66307858]
Média das pontuação da validação cruzada (acurácia):0.667599569429494

AUC probabilidade (Regressão Logística) - Treino: 0.720801236557707

AUC previsão (Regressão Logística) - Treino: 0.6737351991388589
Classificação report do Regressão Logística (Treino):
               precision    recall  f1-score   support

           0       0.66      0.70      0.68      4645
           1       0.69      0.64      0.66      4645

    accuracy                           0.67      9290
   macro avg       0.67      0.67      0.67      9290
weighted avg       0.67      0.67      0.67      9290

Resultado Fase de Teste

AUC probabilidade (Regressão Logística) - Teste: 0.7316666259402868

AUC previsão (Regressão Logística) - Teste: 0.6803654324328218
Matriz de confusão AUC (Regressão Logística) - Teste:
[[4975 2034]
 [ 695 1296]]
Classificação report do Regressão Logística (Te

### XGBoost

In [18]:
def run_XGBoost(X_train, X_test, y_train, y_test):
    xgboost = xgb.XGBClassifier(random_state=0, max_depth=10, learning_rate=0.01, subsample=0.8, colsample_bytree=0.8, 
                                reg_alpha=1, reg_lambda=1)
    xgboost.fit(X_train, y_train)
    score = cross_val_score(xgboost, X_train, y_train, cv=5, scoring='accuracy')
    print('Resultado Fase de Treinamento')
    print("Pontações da validação cruzada (acurácia) {}".format(score))
    print("Média das pontuação da validação cruzada (acurácia):{}".format(score.mean()))
    
    pred_proba_train = xgboost.predict_proba(X_train)[:, 1] # Probabilidade da classe positiva para os dados de treino
    print('\nAUC probabilidade (XGBoost) - Treino: {}'.format(roc_auc_score(y_train, pred_proba_train)))
    pred_class_train = xgboost.predict(X_train) # previsão classe positiva para os dados de treino 
    print('\nAUC previsão (XGBoost) - Treino: {}'.format(roc_auc_score(y_train, pred_class_train)))
    print('Classificação report do XGBoost (Treino):\n {}'.format(classification_report(y_train, pred_class_train)))
    
    print('Resultado Fase de Teste')
    pred_proba_test = xgboost.predict_proba(X_test)[:, 1] # Probabilidade da classe positiva para os dados de teste
    print('\nAUC probabilidade (XGBoost) - Teste: {}'.format(roc_auc_score(y_test, pred_proba_test)))
    pred_class_test = xgboost.predict(X_test) # previsão classe positiva para os dados de teste
    print('\nAUC previsão (XGBoost) - Teste: {}'.format(roc_auc_score(y_test, pred_class_test)))
    print('Matriz de confusão AUC (XGBoost) - Teste:\n{}'.format(confusion_matrix(y_test, pred_class_test)))
    print('Classificação report do XGBoost (Teste):\n {}'.format(classification_report(y_test, pred_class_test)))
    return pred_proba_train, pred_class_train, pred_proba_test, pred_class_test

pred_proba_train, pred_class_train, pred_proba_test, pred_class_test = run_XGBoost(X_train,X_test, y_train, y_test)

Resultado Fase de Treinamento
Pontações da validação cruzada (acurácia) [0.71582347 0.69967707 0.70828848 0.69052745 0.70613563]
Média das pontuação da validação cruzada (acurácia):0.7040904198062432

AUC probabilidade (XGBoost) - Treino: 0.9042433905225822

AUC previsão (XGBoost) - Treino: 0.8299246501614639
Classificação report do XGBoost (Treino):
               precision    recall  f1-score   support

           0       0.80      0.89      0.84      4645
           1       0.87      0.77      0.82      4645

    accuracy                           0.83      9290
   macro avg       0.83      0.83      0.83      9290
weighted avg       0.83      0.83      0.83      9290

Resultado Fase de Teste

AUC probabilidade (XGBoost) - Teste: 0.7799055659155026

AUC previsão (XGBoost) - Teste: 0.7116497774010727
Matriz de confusão AUC (XGBoost) - Teste:
[[5579 1430]
 [ 742 1249]]
Classificação report do XGBoost (Teste):
               precision    recall  f1-score   support

           0       0

### LightGBM 

In [19]:
def run_LightGBM(X_train, X_test, y_train, y_test):
    gbm = lgb.LGBMClassifier(random_state=0, max_depth=10, learning_rate=0.02)
    gbm.fit(X_train, y_train)
    score = cross_val_score(gbm, X_train, y_train, cv=5, scoring='accuracy')
    print('Resultado Fase de Treinamento')
    print("Pontações da validação cruzada (acurácia) {}".format(score))
    print("Média das pontuação da validação cruzada (acurácia):{}".format(score.mean()))
    
    pred_proba_train = gbm.predict_proba(X_train)[:, 1] # Probabilidade da classe positiva para os dados de treino
    print('\nAUC probabilidade (LightGBM) - Treino: {}'.format(roc_auc_score(y_train, pred_proba_train)))
    pred_class_train = gbm.predict(X_train) # previsão classe positiva para os dados de treino 
    print('\nAUC previsão (LightGBM) - Treino: {}'.format(roc_auc_score(y_train, pred_class_train)))
    print('Classificação report do LightGBM (Treino):\n {}'.format(classification_report(y_train, pred_class_train)))
    
    print('Resultado Fase de Teste')
    pred_proba_test = gbm.predict_proba(X_test)[:, 1] # Probabilidade da classe positiva para os dados de teste
    print('\nAUC probabilidade (LightGBM) - Teste: {}'.format(roc_auc_score(y_test, pred_proba_test)))
    pred_class_test = gbm.predict(X_test) # previsão classe positiva para os dados de teste
    print('\nAUC previsão (LightGBM) - Teste: {}'.format(roc_auc_score(y_test, pred_class_test)))
    print('Matriz de confusão AUC (LightGBM) - Teste:\n{}'.format(confusion_matrix(y_test, pred_class_test)))
    print('Classificação report do LightGBM (Teste):\n {}'.format(classification_report(y_test, pred_class_test)))
    return pred_proba_train, pred_class_train, pred_proba_test, pred_class_test

pred_proba_train, pred_class_train, pred_proba_test, pred_class_test = run_LightGBM(X_train,X_test, y_train, y_test)

Resultado Fase de Treinamento
Pontações da validação cruzada (acurácia) [0.71689989 0.70398278 0.71743811 0.69483315 0.70452099]
Média das pontuação da validação cruzada (acurácia):0.7075349838536059

AUC probabilidade (LightGBM) - Treino: 0.829957927838886

AUC previsão (LightGBM) - Treino: 0.7432723358449946
Classificação report do LightGBM (Treino):
               precision    recall  f1-score   support

           0       0.71      0.82      0.76      4645
           1       0.79      0.67      0.72      4645

    accuracy                           0.74      9290
   macro avg       0.75      0.74      0.74      9290
weighted avg       0.75      0.74      0.74      9290

Resultado Fase de Teste

AUC probabilidade (LightGBM) - Teste: 0.7837397336380096

AUC previsão (LightGBM) - Teste: 0.7155762064975081
Matriz de confusão AUC (LightGBM) - Teste:
[[5627 1382]
 [ 740 1251]]
Classificação report do LightGBM (Teste):
               precision    recall  f1-score   support

           0  

### CatBoost

In [20]:
def run_CatBoost(X_train, X_test, y_train, y_test):
    cb = CatBoostClassifier(random_state=0, learning_rate= 0.01, max_depth= 5, subsample= 0.8, verbose=False)
    cb.fit(X_train, y_train)
    score = cross_val_score(cb, X_train, y_train, cv=5, scoring='accuracy')
    print('Resultado Fase de Treinamento')
    print("Pontações da validação cruzada (acurácia) {}".format(score))
    print("Média das pontuação da validação cruzada (acurácia):{}".format(score.mean()))
    
    pred_proba_train = cb.predict_proba(X_train)[:, 1] # Probabilidade da classe positiva para os dados de treino
    print('\nAUC probabilidade (CatBoost) - Treino: {}'.format(roc_auc_score(y_train, pred_proba_train)))
    pred_class_train = cb.predict(X_train) # previsão classe positiva para os dados de treino 
    print('\nAUC previsão (CatBoost) - Treino: {}'.format(roc_auc_score(y_train, pred_class_train)))
    print('Classificação report do CatBoost (Treino):\n {}'.format(classification_report(y_train, pred_class_train)))
    
    print('Resultado Fase de Teste')
    pred_proba_test = cb.predict_proba(X_test)[:, 1] # Probabilidade da classe positiva para os dados de teste
    print('\nAUC probabilidade (CatBoost) - Teste: {}'.format(roc_auc_score(y_test, pred_proba_test)))
    pred_class_test = cb.predict(X_test) # previsão classe positiva para os dados de teste
    print('\nAUC previsão (CatBoost) - Teste: {}'.format(roc_auc_score(y_test, pred_class_test)))
    print('Matriz de confusão AUC (CatBoost) - Teste:\n{}'.format(confusion_matrix(y_test, pred_class_test)))
    print('Classificação report do CatBoost (Teste):\n {}'.format(classification_report(y_test, pred_class_test)))
    return pred_proba_train, pred_class_train, pred_proba_test, pred_class_test

pred_proba_train, pred_class_train, pred_proba_test, pred_class_test = run_CatBoost(X_train,X_test, y_train, y_test)

Resultado Fase de Treinamento
Pontações da validação cruzada (acurácia) [0.72604952 0.71420883 0.71097955 0.69590958 0.7131324 ]
Média das pontuação da validação cruzada (acurácia):0.7120559741657695

AUC probabilidade (CatBoost) - Treino: 0.8183307397910412

AUC previsão (CatBoost) - Treino: 0.7341227125941873
Classificação report do CatBoost (Treino):
               precision    recall  f1-score   support

           0       0.70      0.81      0.75      4645
           1       0.78      0.66      0.71      4645

    accuracy                           0.73      9290
   macro avg       0.74      0.73      0.73      9290
weighted avg       0.74      0.73      0.73      9290

Resultado Fase de Teste

AUC probabilidade (CatBoost) - Teste: 0.7868743989126701

AUC previsão (CatBoost) - Teste: 0.715262446166832
Matriz de confusão AUC (CatBoost) - Teste:
[[5605 1404]
 [ 735 1256]]
Classificação report do CatBoost (Teste):
               precision    recall  f1-score   support

           0  

### Redes Neurais

In [21]:
def run_RNeural(X_train, X_test, y_train, y_test):
    rn = MLPClassifier(max_iter=500,random_state=0)
    rn.fit(X_train, y_train)
    score = cross_val_score(rn, X_train, y_train, cv=5, scoring='accuracy')
    print('Resultado Fase de Treinamento')
    print("Pontações da validação cruzada (acurácia) {}".format(score))
    print("Média das pontuação da validação cruzada (acurácia):{}".format(score.mean()))
    
    pred_proba_train = rn.predict_proba(X_train)[:, 1] # Probabilidade da classe positiva para os dados de treino
    print('\nAUC probabilidade (Redes Neurais) - Treino: {}'.format(roc_auc_score(y_train, pred_proba_train)))
    pred_class_train = rn.predict(X_train) # previsão classe positiva para os dados de treino 
    print('\nAUC previsão (Redes Neurais) - Treino: {}'.format(roc_auc_score(y_train, pred_class_train)))
    print('Classificação report do Redes Neurais (Treino):\n {}'.format(classification_report(y_train, pred_class_train)))
    
    print('Resultado Fase de Teste')
    pred_proba_test = rn.predict_proba(X_test)[:, 1] # Probabilidade da classe positiva para os dados de teste
    print('\nAUC probabilidade (Redes Neurais) - Teste: {}'.format(roc_auc_score(y_test, pred_proba_test)))
    pred_class_test = rn.predict(X_test) # previsão classe positiva positiva para os dados de teste
    print('\nAUC previsão (Redes Neurais) - Teste: {}'.format(roc_auc_score(y_test, pred_class_test)))
    print('Matriz de confusão AUC (Redes Neurais) - Teste:\n{}'.format(confusion_matrix(y_test, pred_class_test)))
    print('Classificação report do Redes Neurais (Teste):\n {}'.format(classification_report(y_test, pred_class_test)))
    return pred_proba_train, pred_class_train, pred_proba_test, pred_class_test

pred_proba_train, pred_class_train, pred_proba_test, pred_class_test = run_RNeural(X_train,X_test, y_train, y_test)

Resultado Fase de Treinamento
Pontações da validação cruzada (acurácia) [0.67922497 0.68891281 0.68299247 0.66361679 0.67653391]
Média das pontuação da validação cruzada (acurácia):0.6782561894510226

AUC probabilidade (Redes Neurais) - Treino: 0.8505683275765578

AUC previsão (Redes Neurais) - Treino: 0.763832077502691
Classificação report do Redes Neurais (Treino):
               precision    recall  f1-score   support

           0       0.74      0.81      0.77      4645
           1       0.79      0.72      0.75      4645

    accuracy                           0.76      9290
   macro avg       0.77      0.76      0.76      9290
weighted avg       0.77      0.76      0.76      9290

Resultado Fase de Teste

AUC probabilidade (Redes Neurais) - Teste: 0.7558841079622176

AUC previsão (Redes Neurais) - Teste: 0.6950143888330702
Matriz de confusão AUC (Redes Neurais) - Teste:
[[5293 1716]
 [ 727 1264]]
Classificação report do Redes Neurais (Teste):
               precision    recall 

# Modelo parcimonioso

Alguns testes permitiu identificar que criar novas variáveis a partir de outras torna o modelo mais simples e reduz significativamente o número de variáveis obtendo um resultado similar. Diante disso, resumi as 18 variáveis que se referem aos valores de faturas, valores pagos e status de pagamento em 3 variáveis, conforme segue. 


In [22]:
df['Status_6M'] = df[['Status_Set','Status_Ago','Status_Jul','Status_Jun','Status_Mai','Status_Abr']].sum(axis=1)
df['Valor_6M'] = df[['Valor_Set','Valor_Ago','Valor_Jul','Valor_Jun','Valor_Mai','Valor_Abr']].sum(axis=1)
df['Pag_6M'] = df[['Pag_Set','Pag_Ago','Pag_Jul','Pag_Jun','Pag_Mai','Pag_Abr']].sum(axis=1)

In [23]:
df

,Limite_Credito,Sexo,Escolaridade,Est_Civil,Idade,Status_Set,Status_Ago,Status_Jul,Status_Jun,Status_Mai,...,Pag_Set,Pag_Ago,Pag_Jul,Pag_Jun,Pag_Mai,Pag_Abr,Status Pagamento,Status_6M,Valor_6M,Pag_6M
0,20000,2,2,1,24,2,2,-1,-1,-2,...,0,689,0,0,0,0,1,-2,7704,689
1,120000,2,2,2,26,-1,2,0,0,0,...,0,1000,1000,1000,0,2000,1,3,17077,5000
2,90000,2,2,2,34,0,0,0,0,0,...,1518,1500,1000,1000,1000,5000,0,0,101653,11018
3,50000,2,2,1,37,0,0,0,0,0,...,2000,2019,1200,1100,1069,1000,0,0,231334,8388
4,50000,1,2,1,57,-1,0,-1,0,0,...,2000,36681,10000,9000,689,679,0,-2,109339,59049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,220000,1,3,1,39,0,0,0,0,0,...,8500,20000,5003,3047,5000,1000,0,0,725349,42550
29996,150000,1,3,2,43,-1,-1,-1,-1,0,...,1837,3526,8998,129,0,0,0,-4,21182,14490
29997,30000,1,2,2,37,4,3,2,-1,0,...,0,0,22000,4200,2000,3100,1,8,70496,31300
29998,80000,1,3,1,41,1,-1,0,0,0,...,85900,3409,1178,1926,52964,1804,1,-1,266611,147181


Seguimos o mesmo processo anterior, divisão de dados entre previsores e classe, divisão entre treino e teste e treinamento dos modelos. 

## Divisão previsores e classe

In [24]:
X_df = df.iloc[:, [0,1,2,3,4,5,24,25,26]].values 
X_df

array([[ 20000,      2,      2, ...,     -2,   7704,    689],
       [120000,      2,      2, ...,      3,  17077,   5000],
       [ 90000,      2,      2, ...,      0, 101653,  11018],
       ...,
       [ 30000,      1,      2, ...,      8,  70496,  31300],
       [ 80000,      1,      3, ...,     -1, 266611, 147181],
       [ 50000,      1,      2, ...,      0, 230874,   8308]], dtype=int64)

O status da fatura de setembro foi utilizada, devido ser uma das variáveis que mais contribuiram com o modelo.

In [25]:
y_df = df.iloc[:, 23].values
y_df

array([1, 1, 0, ..., 1, 1, 1], dtype=int64)

## Escalonamento de variáveis

In [26]:
from sklearn.preprocessing import StandardScaler
scaler_census = StandardScaler()
X_df = scaler_census.fit_transform(X_df)
X_df[0]

array([-1.13672015,  0.81016074,  0.21186989, -1.06879721, -1.24601985,
        1.79456386, -0.15363537, -0.69069198, -0.50902664])

## Divisão entre treinamento e teste

In [27]:
from sklearn.model_selection import train_test_split
seed=1336
np.random.seed(seed)
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size = 0.30, stratify = y_df)

In [28]:
X_train.shape, y_train.shape

((21000, 9), (21000,))

In [29]:
X_test.shape, y_test.shape

((9000, 9), (9000,))

## Técnica de subamostragem da classe majoritária

In [30]:
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
print("Antes: ", Counter(y_train))

Antes:  Counter({0: 16355, 1: 4645})


In [31]:
undersample = RandomUnderSampler(sampling_strategy=1)
X_train, y_train = undersample.fit_resample(X_train, y_train)

In [32]:
print("Depois: ", Counter(y_train))

Depois:  Counter({0: 4645, 1: 4645})


## Modelos de Machine Learning 

Observe que utilizo tanto o *predict* - capacidade do modelo de distinguir entre ambas as classes, quanto o *predict_proba* - capacidade de classificar corretamente a classe positiva, e para ambos avalio usando a *Area Under the Curve* (AUC).

### Random Forest

In [33]:
def run_random_forest(X_train, X_test, y_train, y_test):
    rf = RandomForestClassifier(random_state=0, n_estimators=100, max_depth=10, min_samples_split=5, 
                                min_samples_leaf=8, max_features=9)
    rf.fit(X_train, y_train) #treinando o modelo
    score = cross_val_score(rf, X_train, y_train, cv=5, scoring='accuracy')
    print('Resultado Fase de Treinamento')
    print("Pontações da validação cruzada (acurácia) {}".format(score))
    print("Média das pontuação da validação cruzada (acurácia):{}".format(score.mean()))
    
    pred_proba_train = rf.predict_proba(X_train)[:, 1] # Probabilidade da classe positiva para os dados de treino
    print('\nAUC probabilidade (Random Forest) - Treino: {}'.format(roc_auc_score(y_train, pred_proba_train)))
    pred_class_train = rf.predict(X_train) # previsão classe positiva para os dados de treino 
    print('\nAUC previsão (Random Forest) - Treino: {}'.format(roc_auc_score(y_train, pred_class_train)))
    print('Classificação report do Random Forest (Treino):\n {}'.format(classification_report(y_train, pred_class_train)))
    
    print('Resultado Fase de Teste')
    pred_proba_test = rf.predict_proba(X_test)[:, 1] # Probabilidade da classe positiva para os dados de teste
    print('\nAUC probabilidade (Random Forest) - Teste: {}'.format(roc_auc_score(y_test, pred_proba_test)))
    pred_class_test = rf.predict(X_test) # previsão classe positiva para os dados de teste
    print('\nAUC previsão (Random Forest) - Teste: {}'.format(roc_auc_score(y_test, pred_class_test)))
    print('Matriz de confusão AUC (Random Forest) - Teste:\n{}'.format(confusion_matrix(y_test, pred_class_test)))
    print('Classificação report do Random Forest (Teste):\n {}'.format(classification_report(y_test, pred_class_test)))
    return pred_proba_train, pred_class_train, pred_proba_test, pred_class_test

pred_proba_train, pred_class_train, pred_proba_test, pred_class_test = run_random_forest(X_train,X_test,y_train, y_test)

Resultado Fase de Treinamento
Pontações da validação cruzada (acurácia) [0.71474704 0.69860065 0.70667384 0.69321851 0.7088267 ]
Média das pontuação da validação cruzada (acurácia):0.7044133476856835

AUC probabilidade (Random Forest) - Treino: 0.8679297275563964

AUC previsão (Random Forest) - Treino: 0.7813778256189452
Classificação report do Random Forest (Treino):
               precision    recall  f1-score   support

           0       0.74      0.86      0.80      4645
           1       0.84      0.70      0.76      4645

    accuracy                           0.78      9290
   macro avg       0.79      0.78      0.78      9290
weighted avg       0.79      0.78      0.78      9290

Resultado Fase de Teste

AUC probabilidade (Random Forest) - Teste: 0.7730088938531281

AUC previsão (Random Forest) - Teste: 0.7030939412833568
Matriz de confusão AUC (Random Forest) - Teste:
[[5621 1388]
 [ 788 1203]]
Classificação report do Random Forest (Teste):
               precision    recall

### Regressão Logística

In [34]:
def run_Reg_log(X_train, X_test, y_train, y_test):
    rl = LogisticRegression(random_state=0)
    rl.fit(X_train, y_train) #treinando o modelo
    score = cross_val_score(rl, X_train, y_train, cv=5, scoring='accuracy')
    print('Resultado Fase de Treinamento')
    print("Pontações da validação cruzada (acurácia) {}".format(score))
    print("Média das pontuação da validação cruzada (acurácia):{}".format(score.mean()))
    
    pred_proba_train = rl.predict_proba(X_train)[:, 1] # Probabilidade da classe positiva para os dados de treino
    print('\nAUC probabilidade (Regressão Logística) - Treino: {}'.format(roc_auc_score(y_train, pred_proba_train)))
    pred_class_train = rl.predict(X_train) # previsão classe positiva para os dados de treino 
    print('\nAUC previsão (Regressão Logística) - Treino: {}'.format(roc_auc_score(y_train, pred_class_train)))
    print('Classificação report do Regressão Logística (Treino):\n {}'.format(classification_report(y_train, pred_class_train)))
    
    print('Resultado Fase de Teste')
    pred_proba_test = rl.predict_proba(X_test)[:, 1] # Probabilidade da classe positiva para os dados de teste
    print('\nAUC probabilidade (Regressão Logística) - Teste: {}'.format(roc_auc_score(y_test, pred_proba_test)))
    pred_class_test = rl.predict(X_test) # previsão classe positiva para os dados de teste
    print('\nAUC previsão (Regressão Logística) - Teste: {}'.format(roc_auc_score(y_test, pred_class_test)))
    print('Matriz de confusão AUC (Regressão Logística) - Teste:\n{}'.format(confusion_matrix(y_test, pred_class_test)))
    print('Classificação report do Regressão Logística (Teste):\n {}'.format(classification_report(y_test, pred_class_test)))
    return pred_proba_train, pred_class_train, pred_proba_test, pred_class_test

pred_proba_train, pred_class_train, pred_proba_test, pred_class_test = run_Reg_log(X_train,X_test, y_train, y_test)

Resultado Fase de Treinamento
Pontações da validação cruzada (acurácia) [0.6679225  0.66684607 0.66146394 0.66361679 0.65823466]
Média das pontuação da validação cruzada (acurácia):0.6636167922497309

AUC probabilidade (Regressão Logística) - Treino: 0.7171653490390375

AUC previsão (Regressão Logística) - Treino: 0.6684607104413347
Classificação report do Regressão Logística (Treino):
               precision    recall  f1-score   support

           0       0.66      0.70      0.68      4645
           1       0.68      0.64      0.66      4645

    accuracy                           0.67      9290
   macro avg       0.67      0.67      0.67      9290
weighted avg       0.67      0.67      0.67      9290

Resultado Fase de Teste

AUC probabilidade (Regressão Logística) - Teste: 0.7290668974861122

AUC previsão (Regressão Logística) - Teste: 0.6739648220100741
Matriz de confusão AUC (Regressão Logística) - Teste:
[[4924 2085]
 [ 706 1285]]
Classificação report do Regressão Logística (

### XGBoost

In [35]:
def run_XGBoost(X_train, X_test, y_train, y_test):
    xgboost = xgb.XGBClassifier(random_state=0, max_depth=10, learning_rate=0.01, subsample=0.8, colsample_bytree=0.8, 
                                reg_alpha=1, reg_lambda=1)
    xgboost.fit(X_train, y_train)
    score = cross_val_score(xgboost, X_train, y_train, cv=5, scoring='accuracy')
    print('Resultado Fase de Treinamento')
    print("Pontações da validação cruzada (acurácia) {}".format(score))
    print("Média das pontuação da validação cruzada (acurácia):{}".format(score.mean()))
    
    pred_proba_train = xgboost.predict_proba(X_train)[:, 1] # Probabilidade da classe positiva para os dados de treino
    print('\nAUC probabilidade (XGBoost) - Treino: {}'.format(roc_auc_score(y_train, pred_proba_train)))
    pred_class_train = xgboost.predict(X_train) # previsão classe positiva para os dados de treino 
    print('\nAUC previsão (XGBoost) - Treino: {}'.format(roc_auc_score(y_train, pred_class_train)))
    print('Classificação report do XGBoost (Treino):\n {}'.format(classification_report(y_train, pred_class_train)))
    
    print('Resultado Fase de Teste')
    pred_proba_test = xgboost.predict_proba(X_test)[:, 1] # Probabilidade da classe positiva para os dados de teste
    print('\nAUC probabilidade (XGBoost) - Teste: {}'.format(roc_auc_score(y_test, pred_proba_test)))
    pred_class_test = xgboost.predict(X_test) # previsão classe positiva para os dados de teste
    print('\nAUC previsão (XGBoost) - Teste: {}'.format(roc_auc_score(y_test, pred_class_test)))
    print('Matriz de confusão AUC (XGBoost) - Teste:\n{}'.format(confusion_matrix(y_test, pred_class_test)))
    print('Classificação report do XGBoost (Teste):\n {}'.format(classification_report(y_test, pred_class_test)))
    return pred_proba_train, pred_class_train, pred_proba_test, pred_class_test

pred_proba_train, pred_class_train, pred_proba_test, pred_class_test = run_XGBoost(X_train,X_test, y_train, y_test)

Resultado Fase de Treinamento
Pontações da validação cruzada (acurácia) [0.71420883 0.70775027 0.70667384 0.69375673 0.7131324 ]
Média das pontuação da validação cruzada (acurácia):0.7071044133476857

AUC probabilidade (XGBoost) - Treino: 0.8672390535327986

AUC previsão (XGBoost) - Treino: 0.7919268030139935
Classificação report do XGBoost (Treino):
               precision    recall  f1-score   support

           0       0.76      0.86      0.81      4645
           1       0.84      0.72      0.78      4645

    accuracy                           0.79      9290
   macro avg       0.80      0.79      0.79      9290
weighted avg       0.80      0.79      0.79      9290

Resultado Fase de Teste

AUC probabilidade (XGBoost) - Teste: 0.7749868343915146

AUC previsão (XGBoost) - Teste: 0.7064160673379759
Matriz de confusão AUC (XGBoost) - Teste:
[[5569 1440]
 [ 760 1231]]
Classificação report do XGBoost (Teste):
               precision    recall  f1-score   support

           0       0

### LightGBM 

In [36]:
def run_LightGBM(X_train, X_test, y_train, y_test):
    gbm = lgb.LGBMClassifier(random_state=0, max_depth=10, learning_rate=0.02)
    gbm.fit(X_train, y_train)
    score = cross_val_score(gbm, X_train, y_train, cv=5, scoring='accuracy')
    print('Resultado Fase de Treinamento')
    print("Pontações da validação cruzada (acurácia) {}".format(score))
    print("Média das pontuação da validação cruzada (acurácia):{}".format(score.mean()))
    
    pred_proba_train = gbm.predict_proba(X_train)[:, 1] # Probabilidade da classe positiva para os dados de treino
    print('\nAUC probabilidade (LightGBM) - Treino: {}'.format(roc_auc_score(y_train, pred_proba_train)))
    pred_class_train = gbm.predict(X_train) # previsão classe positiva para os dados de treino 
    print('\nAUC previsão (LightGBM) - Treino: {}'.format(roc_auc_score(y_train, pred_class_train)))
    print('Classificação report do LightGBM (Treino):\n {}'.format(classification_report(y_train, pred_class_train)))
    
    print('Resultado Fase de Teste')
    pred_proba_test = gbm.predict_proba(X_test)[:, 1] # Probabilidade da classe positiva para os dados de teste
    print('\nAUC probabilidade (LightGBM) - Teste: {}'.format(roc_auc_score(y_test, pred_proba_test)))
    pred_class_test = gbm.predict(X_test) # previsão classe positiva para os dados de teste
    print('\nAUC previsão (LightGBM) - Teste: {}'.format(roc_auc_score(y_test, pred_class_test)))
    print('Matriz de confusão AUC (LightGBM) - Teste:\n{}'.format(confusion_matrix(y_test, pred_class_test)))
    print('Classificação report do LightGBM (Teste):\n {}'.format(classification_report(y_test, pred_class_test)))
    return pred_proba_train, pred_class_train, pred_proba_test, pred_class_test

pred_proba_train, pred_class_train, pred_proba_test, pred_class_test = run_LightGBM(X_train,X_test, y_train, y_test)

Resultado Fase de Treinamento
Pontações da validação cruzada (acurácia) [0.71474704 0.7007535  0.70721206 0.69752422 0.71097955]
Média das pontuação da validação cruzada (acurácia):0.7062432723358449

AUC probabilidade (LightGBM) - Treino: 0.8131806947757986

AUC previsão (LightGBM) - Treino: 0.735091496232508
Classificação report do LightGBM (Treino):
               precision    recall  f1-score   support

           0       0.70      0.81      0.75      4645
           1       0.78      0.66      0.71      4645

    accuracy                           0.74      9290
   macro avg       0.74      0.74      0.73      9290
weighted avg       0.74      0.74      0.73      9290

Resultado Fase de Teste

AUC probabilidade (LightGBM) - Teste: 0.7770132882892405

AUC previsão (LightGBM) - Teste: 0.7089157593820502
Matriz de confusão AUC (LightGBM) - Teste:
[[5597 1412]
 [ 758 1233]]
Classificação report do LightGBM (Teste):
               precision    recall  f1-score   support

           0  

### CatBoost

In [37]:
def run_CatBoost(X_train, X_test, y_train, y_test):
    cb = CatBoostClassifier(random_state=0, learning_rate= 0.01, max_depth= 5, subsample= 0.8, verbose=False)
    cb.fit(X_train, y_train)
    score = cross_val_score(cb, X_train, y_train, cv=5, scoring='accuracy')
    print('Resultado Fase de Treinamento')
    print("Pontações da validação cruzada (acurácia) {}".format(score))
    print("Média das pontuação da validação cruzada (acurácia):{}".format(score.mean()))
    
    pred_proba_train = cb.predict_proba(X_train)[:, 1] # Probabilidade da classe positiva para os dados de treino
    print('\nAUC probabilidade (CatBoost) - Treino: {}'.format(roc_auc_score(y_train, pred_proba_train)))
    pred_class_train = cb.predict(X_train) # previsão classe positiva para os dados de treino 
    print('\nAUC previsão (CatBoost) - Treino: {}'.format(roc_auc_score(y_train, pred_class_train)))
    print('Classificação report do CatBoost (Treino):\n {}'.format(classification_report(y_train, pred_class_train)))
    
    print('Resultado Fase de Teste')
    pred_proba_test = cb.predict_proba(X_test)[:, 1] # Probabilidade da classe positiva para os dados de teste
    print('\nAUC probabilidade (CatBoost) - Teste: {}'.format(roc_auc_score(y_test, pred_proba_test)))
    pred_class_test = cb.predict(X_test) # previsão classe positiva para os dados de teste
    print('\nAUC previsão (CatBoost) - Teste: {}'.format(roc_auc_score(y_test, pred_class_test)))
    print('Matriz de confusão AUC (CatBoost) - Teste:\n{}'.format(confusion_matrix(y_test, pred_class_test)))
    print('Classificação report do CatBoost (Teste):\n {}'.format(classification_report(y_test, pred_class_test)))
    return pred_proba_train, pred_class_train, pred_proba_test, pred_class_test

pred_proba_train, pred_class_train, pred_proba_test, pred_class_test = run_CatBoost(X_train,X_test, y_train, y_test)

Resultado Fase de Treinamento
Pontações da validação cruzada (acurácia) [0.72066738 0.70775027 0.7088267  0.70021529 0.71689989]
Média das pontuação da validação cruzada (acurácia):0.7108719052744886

AUC probabilidade (CatBoost) - Treino: 0.8065042796344554

AUC previsão (CatBoost) - Treino: 0.7289558665231431
Classificação report do CatBoost (Treino):
               precision    recall  f1-score   support

           0       0.70      0.81      0.75      4645
           1       0.78      0.64      0.70      4645

    accuracy                           0.73      9290
   macro avg       0.74      0.73      0.73      9290
weighted avg       0.74      0.73      0.73      9290

Resultado Fase de Teste

AUC probabilidade (CatBoost) - Teste: 0.7817700339213721

AUC previsão (CatBoost) - Teste: 0.7154219239825039
Matriz de confusão AUC (CatBoost) - Teste:
[[5653 1356]
 [ 748 1243]]
Classificação report do CatBoost (Teste):
               precision    recall  f1-score   support

           0 

### Redes Neurais

In [38]:
def run_RNeural(X_train, X_test, y_train, y_test):
    rn = MLPClassifier(max_iter=500,random_state=0)
    rn.fit(X_train, y_train)
    score = cross_val_score(rn, X_train, y_train, cv=5, scoring='accuracy')
    print('Resultado Fase de Treinamento')
    print("Pontações da validação cruzada (acurácia) {}".format(score))
    print("Média das pontuação da validação cruzada (acurácia):{}".format(score.mean()))
    
    pred_proba_train = rn.predict_proba(X_train)[:, 1] # Probabilidade da classe positiva para os dados de treino
    print('\nAUC probabilidade (Redes Neurais) - Treino: {}'.format(roc_auc_score(y_train, pred_proba_train)))
    pred_class_train = rn.predict(X_train) # previsão classe positiva para os dados de treino 
    print('\nAUC previsão (Redes Neurais) - Treino: {}'.format(roc_auc_score(y_train, pred_class_train)))
    print('Classificação report do Redes Neurais (Treino):\n {}'.format(classification_report(y_train, pred_class_train)))
    
    print('Resultado Fase de Teste')
    pred_proba_test = rn.predict_proba(X_test)[:, 1] # Probabilidade da classe positiva para os dados de teste
    print('\nAUC probabilidade (Redes Neurais) - Teste: {}'.format(roc_auc_score(y_test, pred_proba_test)))
    pred_class_test = rn.predict(X_test) # previsão classe positiva positiva para os dados de teste
    print('\nAUC previsão (Redes Neurais) - Teste: {}'.format(roc_auc_score(y_test, pred_class_test)))
    print('Matriz de confusão AUC (Redes Neurais) - Teste:\n{}'.format(confusion_matrix(y_test, pred_class_test)))
    print('Classificação report do Redes Neurais (Teste):\n {}'.format(classification_report(y_test, pred_class_test)))
    return pred_proba_train, pred_class_train, pred_proba_test, pred_class_test

pred_proba_train, pred_class_train, pred_proba_test, pred_class_test = run_RNeural(X_train,X_test, y_train, y_test)

Resultado Fase de Treinamento
Pontações da validação cruzada (acurácia) [0.69967707 0.70398278 0.69752422 0.68675996 0.69806243]
Média das pontuação da validação cruzada (acurácia):0.6972012917115178

AUC probabilidade (Redes Neurais) - Treino: 0.8122566830544551

AUC previsão (Redes Neurais) - Treino: 0.7298170075349838
Classificação report do Redes Neurais (Treino):
               precision    recall  f1-score   support

           0       0.71      0.78      0.74      4645
           1       0.76      0.68      0.71      4645

    accuracy                           0.73      9290
   macro avg       0.73      0.73      0.73      9290
weighted avg       0.73      0.73      0.73      9290

Resultado Fase de Teste

AUC probabilidade (Redes Neurais) - Teste: 0.7676576625059592

AUC previsão (Redes Neurais) - Teste: 0.7019944006840885
Matriz de confusão AUC (Redes Neurais) - Teste:
[[5331 1678]
 [ 710 1281]]
Classificação report do Redes Neurais (Teste):
               precision    recall

Tabelas com o resumo dos resultados referente o modelo com todas as variáveis usando a técnica e subamostragem e o modelo parcimonioso.

### Modelo todas as variáveis (subamostragem)

In [39]:
resultados = {'Random Forest': {'Accuracy': 0.77,'Precision': 0.48,'Recall': 0.62,'AUC probability': 0.78,'AUC predict': 0.72},
        'Regressão Logística': {'Accuracy': 0.70,'Precision': 0.39,'Recall': 0.65,'AUC probability': 0.73,'AUC predict': 0.68},
              'XGBoost': {'Accuracy': 0.76,'Precision': 0.47,'Recall': 0.63,'AUC probability': 0.78,'AUC predict': 0.71},
             'LightGBM': {'Accuracy': 0.76,'Precision': 0.48,'Recall': 0.63,'AUC probability': 0.78,'AUC predict': 0.71},
             'CatBoost': {'Accuracy': 0.76,'Precision': 0.47,'Recall': 0.63,'AUC probability': 0.79,'AUC predict': 0.71},
             'Rede Neural': {'Accuracy': 0.73,'Precision': 0.42,'Recall': 0.63,'AUC probability': 0.76,'AUC predict': 0.69}}

df = pd.DataFrame(resultados).T
df = df.applymap('{:.2f}'.format)
df

,Accuracy,Precision,Recall,AUC probability,AUC predict
Random Forest,0.77,0.48,0.62,0.78,0.72
Regressão Logística,0.70,0.39,0.65,0.73,0.68
XGBoost,0.76,0.47,0.63,0.78,0.71
LightGBM,0.76,0.48,0.63,0.78,0.71
CatBoost,0.76,0.47,0.63,0.79,0.71
Rede Neural,0.73,0.42,0.63,0.76,0.69


### Modelo parcimonioso (subamostragem)

In [40]:
resultados = {'Random Forest': {'Accuracy': 0.76,'Precision': 0.46,'Recall': 0.60,'AUC probability': 0.77,'AUC predict': 0.70},
        'Regressão Logística': {'Accuracy': 0.69,'Precision': 0.38,'Recall': 0.65,'AUC probability': 0.73,'AUC predict': 0.67},
              'XGBoost': {'Accuracy': 0.76,'Precision': 0.46,'Recall': 0.62,'AUC probability': 0.77,'AUC predict': 0.70},
             'LightGBM': {'Accuracy': 0.76,'Precision': 0.47,'Recall': 0.62,'AUC probability': 0.78,'AUC predict': 0.71},
             'CatBoost': {'Accuracy': 0.77,'Precision': 0.48,'Recall': 0.62,'AUC probability': 0.78,'AUC predict': 0.71},
             'Rede Neural': {'Accuracy': 0.73,'Precision': 0.43,'Recall': 0.64,'AUC probability': 0.77,'AUC predict': 0.70}}

df = pd.DataFrame(resultados).T
df = df.applymap('{:.2f}'.format)
df

,Accuracy,Precision,Recall,AUC probability,AUC predict
Random Forest,0.76,0.46,0.60,0.77,0.70
Regressão Logística,0.69,0.38,0.65,0.73,0.67
XGBoost,0.76,0.46,0.62,0.77,0.70
LightGBM,0.76,0.47,0.62,0.78,0.71
CatBoost,0.77,0.48,0.62,0.78,0.71
Rede Neural,0.73,0.43,0.64,0.77,0.70
